<a href="https://colab.research.google.com/github/yiiyama/qc-workbook-lecturenotes/blob/branch-2024/2024_07_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

runtime_config_path = '/content/gdrive/MyDrive/qiskit-ibm.json'

## QuTiPシミュレーション

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qtp

twopi = 2. * np.pi
hbar = 1.

In [ ]:
# 4準位系の|0>状態
qtp.basis(4, 0)

In [ ]:
# 5準位系の生成演算子
qtp.create(5)

In [ ]:
# 5準位系の消滅演算子
qtp.destroy(5)

In [ ]:
# 3準位系の数演算子(a^†a = Σ_j j |j><j|)
qtp.num(3)

In [ ]:
# N = a^†a
qtp.num(4) == qtp.create(4) * qtp.destroy(4)

### 調和振動子（20準位まで）にDC外場

With
$$
H = H_0 + H_d
$$
$$
H_0 = \hbar \omega \sum_{j=0}^{19} j \lvert j \rangle \langle j \rvert
$$
$$
H_d = i D_0 \left(a - a^{\dagger} \right)
$$

Simulate
$$
\lvert \psi (t) \rangle = U(t) \lvert 0 \rangle = \exp \left(-i H t \right) \lvert 0 \rangle
$$

In [ ]:
num_levels = 20
omega = twopi
amp = 0.2
# Free Hamiltonian
h0 = hbar * omega * qtp.num(num_levels)
# Drive Hamiltonian
hdrive = 1.j * amp * (qtp.destroy(num_levels) - qtp.create(num_levels))

In [ ]:
# 初期状態を|0>にする
psi0 = qtp.basis(num_levels, 0)
# 0から100/hbarまでの時間を1000点に区切ってシミュレートする
tlist = np.linspace(0., 20., 1000)
# シミュレーションの実行
result = qtp.sesolve([h0, hdrive], psi0, tlist)
# 各時刻での状態ベクトルを配列に格納
states = np.squeeze(list(state.full() for state in result.states))

In [ ]:
plt.plot(tlist, np.square(np.abs(states))[:, :5], label=[fr'$|\langle {i} | U(t) | 0 \rangle|^2$' for i in range(5)])
plt.legend();

### 調和振動子（20準位まで）の共鳴ドライブ

With
$$
H(t) = H_0 + H_d(t)
$$
$$
H_0 = \hbar \omega \sum_{j=0}^{19} j \lvert j \rangle \langle j \rvert
$$
$$
H_d(t) = i D_0 \sin (\omega t) \left(a - a^{\dagger} \right)
$$

Simulate
$$
\lvert \psi (t) \rangle = U(t) \lvert 0 \rangle = T\left[ \exp \left(-i \int_0^{t} H(t')dt' \right) \right] \lvert 0 \rangle
$$

In [ ]:
num_levels = 20
omega = twopi
amp = 0.2
omega_d = twopi

# Free Hamiltonian
h0 = hbar * omega * qtp.num(num_levels)
# Drive Hamiltonian
hdrive = [qtp.destroy(num_levels) - qtp.create(num_levels), f'1.j * {amp} * sin({omega_d} * t)']

In [ ]:
# 初期状態を|0>にする
psi0 = qtp.basis(num_levels, 0)
# 0から100/hbarまでの時間を1000点に区切ってシミュレートする
tlist = np.linspace(0., 50., 1000)
# シミュレーションの実行
result = qtp.sesolve([h0, hdrive], psi0, tlist)
# 各時刻での状態ベクトルを配列に格納
states = np.squeeze(list(state.full() for state in result.states))

In [ ]:
plt.plot(tlist, np.square(np.abs(states))[:, :5], label=[fr'$|\langle {i} | U(t) | 0 \rangle|^2$' for i in range(5)])
plt.legend();

### 非調和振動子（20準位まで）のドライブ

With
$$
H(t) = H_0 + H_d(t)
$$
$$
H_0 = \hbar \omega \sum_{j=0}^{19} j \left[1 - (j-1) \frac{\alpha}{2}\right]\lvert j \rangle \langle j \rvert
$$
$$
H_d(t) = i D_0 \sin (\omega t) \left(a - a^{\dagger} \right)
$$

Simulate
$$
\lvert \psi (t) \rangle = U(t) \lvert 0 \rangle = T\left[ \exp \left(-i \int_0^{t} H(t')dt' \right) \right] \lvert 0 \rangle
$$

In [ ]:
num_levels = 20
omega = twopi
alpha = -0.06 # anharmonicity
amp = 0.05
omega_d = twopi
# Free Hamiltonian: Σ_j hbar ω j (1 + (j-1)α/2) |j><j|
h0 = hbar * omega * sum(j * (1 + (j - 1) * alpha / 2.) * qtp.basis(num_levels, j) * qtp.basis(num_levels, j).dag()
                        for j in range(num_levels))
# Drive Hamiltonian
hdrive = [qtp.destroy(num_levels) - qtp.create(num_levels), f'1.j * {amp} * sin({omega_d} * t)']

In [ ]:
psi0 = qtp.basis(num_levels, 0)
tlist = np.linspace(0., 100., 1000)
result = qtp.sesolve([h0, hdrive], psi0, tlist)
states = np.squeeze(list(state.full() for state in result.states))

In [ ]:
plt.plot(tlist, np.square(np.abs(states))[:, :5], label=[fr'$|\langle {i} | U(t) | 0 \rangle|^2$' for i in range(5)])
plt.legend();

### Xゲートの実装

パルス信号を$\lvert 0 \rangle$に適用して、終状態が$\lvert 1 \rangle$に比例するようにパラメータを調整しましょう。

In [ ]:
num_levels = 5
omega = twopi
alpha = -0.06 # anharmonicity
amp = 0.05
omega_d = twopi
sigma = 30.

def pulse_amp(t, args=None):
    center = 2. * sigma
    pedestal = np.exp(-2.)
    gauss = np.exp(-np.square((t - center) / sigma) * 0.5)
    envelope = amp * (gauss - pedestal) / (1. - pedestal)
    return 1.j * envelope * np.sin(omega_d * t)

# Free Hamiltonian: Σ_j hbar ω j (1 + (j-1)α/2) |j><j|
h0 = hbar * omega * sum(j * (1 + (j - 1) * alpha / 2.) * qtp.basis(num_levels, j) * qtp.basis(num_levels, j).dag()
                        for j in range(num_levels))
# Drive Hamiltonian
hdrive = [qtp.destroy(num_levels) - qtp.create(num_levels), pulse_amp]

In [ ]:
psi0 = qtp.basis(num_levels, 0)
tlist = np.linspace(0., 4. * sigma, 1000)
result = qtp.sesolve([h0, hdrive], psi0, tlist)
states = np.squeeze(list(state.full() for state in result.states))

In [ ]:
plt.plot(tlist, (-1.j * pulse_amp(tlist)).real)

In [ ]:
plt.plot(tlist, np.square(np.abs(states))[:, :2], label=[fr'$|\langle {i} | U(t) | 0 \rangle|^2$' for i in range(2)])
plt.legend();

## Qiskit pulseの利用

In [ ]:
import scipy.optimize as sciopt
from qiskit import QuantumCircuit, pulse, transpile
from qiskit.circuit import Gate, Parameter
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler
%matplotlib inline

In [ ]:
service = QiskitRuntimeService(filename=runtime_config_path)
#backend = service.backend('ibm_kawasaki')
#physical_qubit = 70
backend = service.backend('ibm_strasbourg')
physical_qubit = 38
session = Session(service=service, backend=backend)

### 回路を実行して"1"が得られる確率を返す関数

In [ ]:
def execute(backend, physical_qubit, circuits, shots=10000, session=None):
    """Add measurements to the circuits and execute them on a sampler."""
    for circuit in circuits:
        circuit.measure_all()

    circuits = transpile(circuits, backend=backend, initial_layout=[physical_qubit], optimization_level=0)
    if session:
        sampler = Sampler(session=session)
    else:
        sampler = Sampler(backend=backend)

    result = sampler.run(circuits, shots=shots).result()
    counts = [res.data.meas.get_counts() for res in result]
    yval = np.array([c.get('1', 0) / shots for c in counts])

    return yval

### カーブをフィットする関数

In [ ]:
def fit(curve, xval, yval, p0, shots):
    """Call scipy curve_fit and make a plot."""
    popt, _ = sciopt.curve_fit(curve, xval, yval, p0=p0)

    plt.errorbar(xval, yval, yerr=np.sqrt(yval * (1. - yval) / shots), fmt='o', markersize=3)
    xfine = np.linspace(xval[0], xval[-1], 400)
    plt.plot(xfine, curve(xfine, *popt))

    return popt

### 共鳴周波数の探索（スペクトロスコピー）

In [ ]:
def rough_frequency_calibration(backend, physical_qubit, shots=10000, session=None):
    """Perform a spectroscopy experiment to find the qubit frequency."""
    frequency = Parameter('frequency')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='spectrocopy') as spectroscopy_sched:
        pulse.shift_frequency(frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=512, amp=0.01, sigma=128), drive_channel)

    frequencies = np.linspace(-1.e+8, 1.e+8, 41)

    circuits = []
    for freq_value in frequencies:
        sched = spectroscopy_sched.assign_parameters({frequency: freq_value}, inplace=False)

        circuit = QuantumCircuit(1)
        circuit.append(Gate('spectroscopy', 1, []), [0])

        circuit.add_calibration('spectroscopy', [physical_qubit], sched)

        circuits.append(circuit)

    yval = execute(backend, physical_qubit, circuits, shots=shots, session=session)

    def resonance_curve(x, f0, a, b, sigma):
        return a * np.exp(-(x - f0)**2 / 2. / sigma**2) + b

    ipeak = len(frequencies) // 2
    f0_guess = frequencies[ipeak]
    a_guess = yval[ipeak] - yval[0]
    b_guess = yval[0]
    half_max = np.nonzero(yval[ipeak:] - (yval[ipeak] / 2) < 0)[0][0] + ipeak
    sigma_guess = frequencies[half_max] - frequencies[ipeak]

    p0 = (f0_guess, a_guess, b_guess, sigma_guess)

    popt = fit(resonance_curve, frequencies, yval, p0, shots)

    qubit_frequency = popt[0] + backend.target.qubit_properties[physical_qubit].frequency

    return qubit_frequency

In [ ]:
qubit_frequency = rough_frequency_calibration(backend, physical_qubit, session=session)

### Xゲートのパルス強度探索

In [ ]:
def rough_amplitude_calibration(backend, physical_qubit, qubit_frequency, shots=10000, session=None):
    """Observe the Rabi oscillation pattern by scanning the Rx pulse amplitude."""
    amp = Parameter('amp')
    drive_channel = pulse.DriveChannel(physical_qubit)
    with pulse.build(name='rabi') as rabi_sched:
        pulse.set_frequency(qubit_frequency, drive_channel)
        pulse.play(pulse.Gaussian(duration=160, amp=amp, sigma=40), drive_channel)

    amplitudes = np.linspace(0., 0.2, 40)

    circuits = []
    for amp_val in amplitudes:
        circuit = QuantumCircuit(1)
        circuit.append(Gate('rabi', 1, []), [0])

        circuit.add_calibration('rabi', [physical_qubit],
                                rabi_sched.assign_parameters({amp: amp_val}, inplace=False))

        circuits.append(circuit)

    yval = execute(backend, physical_qubit, circuits, shots=shots, session=session)

    def oscillation_curve(x, freq, phase, a, b):
        return a * np.cos(x * freq + phase) + b

    indices = np.nonzero(yval > 0.9)[0]
    if len(indices):
        first_above_09 = indices[0]
        indices = np.nonzero(yval[first_above_09:] < 0.1)[0]
        if len(indices):
            next_below_01 = [0] + first_above_09
            first_max = np.argmax(yval[:next_below_01])
            freq_guess = np.pi / amplitudes[first_max]
        else:
            freq_guess = np.pi / 2. / amplitudes[first_above_09]
    else:
        freq_guess = 0.1 * np.pi / amplitudes[-1]
    phase_guess = 0.
    a_guess = -0.5
    b_guess = 0.5

    p0 = (freq_guess, phase_guess, a_guess, b_guess)

    popt = fit(oscillation_curve, amplitudes, yval, p0, shots)

    x_amp = np.pi / popt[0]
    sx_amp = x_amp / 2.

    return x_amp, sx_amp

In [ ]:
x_amp, sx_amp = rough_amplitude_calibration(backend, physical_qubit, qubit_frequency, session=session)

In [ ]:
x_amp_ibm = backend.target['x'][(physical_qubit,)].calibration.instructions[0][1].pulse.amp
print(x_amp, x_amp_ibm)

In [ ]:
session.close()